In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports loaded")

## 1. Configuration & Market Parameters

In [ ]:
@dataclass
class MarketConditions:
    """Market microstructure parameters."""
    # Liquidity
    bid_ask_spread_bps: float = 1.0  # Typical spread in bps
    order_book_depth_usd: float = 5_000_000  # Depth at top 5 levels
    daily_volume_usd: float = 1_000_000_000  # 24h volume
    
    # Funding
    avg_funding_rate_8h: float = 0.0003  # 0.03% per 8h (typical)
    funding_rate_std: float = 0.0005  # Standard deviation
    
    # Execution
    maker_fee: float = 0.0002  # 2 bps
    taker_fee: float = 0.0005  # 5 bps
    
    # Risk
    margin_requirement: float = 0.10  # 10x leverage = 10% margin
    liquidation_buffer: float = 0.05  # 5% safety buffer

@dataclass
class CapacityResult:
    """Results from capacity analysis."""
    position_size_usd: float
    expected_return_annual: float
    slippage_cost_bps: float
    net_return_annual: float
    sharpe_ratio: float
    capital_required: float
    break_even_funding: float
    capacity_utilization: float

# Default market conditions (BTC perp)
BTC_MARKET = MarketConditions(
    bid_ask_spread_bps=0.5,
    order_book_depth_usd=10_000_000,
    daily_volume_usd=5_000_000_000,
    avg_funding_rate_8h=0.0003,
    funding_rate_std=0.0005,
    maker_fee=0.0002,
    taker_fee=0.0005,
    margin_requirement=0.05,  # 20x max
    liquidation_buffer=0.03
)

ETH_MARKET = MarketConditions(
    bid_ask_spread_bps=0.8,
    order_book_depth_usd=5_000_000,
    daily_volume_usd=2_000_000_000,
    avg_funding_rate_8h=0.00035,
    funding_rate_std=0.0006,
    maker_fee=0.0002,
    taker_fee=0.0005,
    margin_requirement=0.05,
    liquidation_buffer=0.03
)

ALTCOIN_MARKET = MarketConditions(
    bid_ask_spread_bps=3.0,
    order_book_depth_usd=500_000,
    daily_volume_usd=100_000_000,
    avg_funding_rate_8h=0.0005,
    funding_rate_std=0.001,
    maker_fee=0.0002,
    taker_fee=0.0005,
    margin_requirement=0.10,
    liquidation_buffer=0.05
)

print("✅ Market configurations defined")
print(f"   BTC: {BTC_MARKET.daily_volume_usd/1e9:.1f}B daily volume, {BTC_MARKET.avg_funding_rate_8h:.2%}/8h funding")
print(f"   ETH: {ETH_MARKET.daily_volume_usd/1e9:.1f}B daily volume, {ETH_MARKET.avg_funding_rate_8h:.2%}/8h funding")
print(f"   ALT: {ALTCOIN_MARKET.daily_volume_usd/1e6:.0f}M daily volume, {ALTCOIN_MARKET.avg_funding_rate_8h:.2%}/8h funding")

## 2. Slippage Model

In [ ]:
class SlippageModel:
    """
    Models market impact and slippage for different order sizes.
    Uses square-root market impact model.
    """
    
    def __init__(self, market: MarketConditions):
        self.market = market
        
    def estimate_slippage_bps(self, order_size_usd: float) -> float:
        """
        Estimate execution slippage in basis points.
        Uses square-root impact model: slippage ∝ √(order_size / ADV)
        """
        # Base spread cost
        spread_cost = self.market.bid_ask_spread_bps / 2
        
        # Market impact (square-root model)
        participation_rate = order_size_usd / self.market.daily_volume_usd
        impact_coefficient = 0.1  # Calibrated for crypto
        market_impact = impact_coefficient * np.sqrt(participation_rate) * 10000  # Convert to bps
        
        # Order book depletion impact
        if order_size_usd > self.market.order_book_depth_usd:
            depth_impact = (order_size_usd / self.market.order_book_depth_usd - 1) * 5  # 5 bps per book depth
        else:
            depth_impact = 0
        
        total_slippage = spread_cost + market_impact + depth_impact
        return total_slippage
    
    def estimate_entry_exit_cost_bps(self, order_size_usd: float, use_maker: bool = True) -> float:
        """
        Total round-trip cost (entry + exit).
        """
        slippage = self.estimate_slippage_bps(order_size_usd)
        
        if use_maker:
            fee_cost = self.market.maker_fee * 10000 * 2  # Round trip
        else:
            fee_cost = self.market.taker_fee * 10000 * 2
        
        return slippage * 2 + fee_cost  # Entry + exit slippage

# Test slippage model
btc_slippage = SlippageModel(BTC_MARKET)
test_sizes = [100_000, 500_000, 1_000_000, 5_000_000, 10_000_000]

print("📊 BTC Slippage Estimates:")
print("-" * 50)
for size in test_sizes:
    slip = btc_slippage.estimate_slippage_bps(size)
    round_trip = btc_slippage.estimate_entry_exit_cost_bps(size)
    print(f"   ${size/1e6:.1f}M order: {slip:.2f} bps slippage, {round_trip:.2f} bps round-trip")

## 3. Funding Arbitrage Return Model

In [ ]:
class FundingArbCapacityAnalyzer:
    """
    Analyzes capacity limits for funding arbitrage.
    """
    
    def __init__(self, market: MarketConditions):
        self.market = market
        self.slippage_model = SlippageModel(market)
        
    def calculate_gross_return(self, holding_periods: int = 3 * 365) -> float:
        """
        Calculate gross annual return from funding payments.
        holding_periods: number of 8-hour periods per year (3 * 365 = 1095)
        """
        # Expected funding capture
        expected_funding = self.market.avg_funding_rate_8h * holding_periods
        return expected_funding
    
    def analyze_capacity(
        self,
        position_sizes_usd: List[float],
        holding_period_days: int = 7,  # Average hold time
        use_maker_orders: bool = True
    ) -> List[CapacityResult]:
        """
        Analyze returns at different position sizes.
        """
        results = []
        
        # Annual funding periods
        funding_periods_per_year = 3 * 365  # 8-hour periods
        trades_per_year = 365 / holding_period_days
        
        for position_size in position_sizes_usd:
            # Gross return
            gross_return = self.calculate_gross_return(funding_periods_per_year)
            
            # Entry/exit costs (amortized over holding period)
            round_trip_cost_bps = self.slippage_model.estimate_entry_exit_cost_bps(
                position_size, use_maker_orders
            )
            annual_trading_cost = round_trip_cost_bps / 10000 * trades_per_year
            
            # Net return
            net_return = gross_return - annual_trading_cost
            
            # Risk metrics
            # Funding volatility creates return volatility
            funding_vol = self.market.funding_rate_std * np.sqrt(funding_periods_per_year)
            sharpe = net_return / funding_vol if funding_vol > 0 else 0
            
            # Capital required (including margin + buffer)
            margin_rate = self.market.margin_requirement + self.market.liquidation_buffer
            capital_required = position_size * margin_rate * 2  # 2x for spot + perp
            
            # Break-even funding rate
            break_even = annual_trading_cost / funding_periods_per_year
            
            # Capacity utilization (vs order book depth)
            capacity_util = position_size / self.market.order_book_depth_usd
            
            results.append(CapacityResult(
                position_size_usd=position_size,
                expected_return_annual=gross_return,
                slippage_cost_bps=round_trip_cost_bps,
                net_return_annual=net_return,
                sharpe_ratio=sharpe,
                capital_required=capital_required,
                break_even_funding=break_even,
                capacity_utilization=capacity_util
            ))
        
        return results
    
    def find_kill_capacity(self, tolerance: float = 0.001) -> float:
        """
        Find position size where net return goes to zero.
        Binary search for "kill" capacity.
        """
        low = 10_000
        high = self.market.daily_volume_usd
        
        while high - low > tolerance * high:
            mid = (low + high) / 2
            result = self.analyze_capacity([mid])[0]
            
            if result.net_return_annual > 0:
                low = mid
            else:
                high = mid
        
        return low

print("✅ Capacity analyzer defined")

## 4. Run Capacity Analysis

In [ ]:
# Analyze BTC funding arb capacity
btc_analyzer = FundingArbCapacityAnalyzer(BTC_MARKET)

# Position sizes to test
position_sizes = np.logspace(4, 8, 50)  # $10K to $100M

btc_results = btc_analyzer.analyze_capacity(position_sizes.tolist())

# Convert to DataFrame
btc_df = pd.DataFrame([
    {
        'Position Size ($M)': r.position_size_usd / 1e6,
        'Gross Return (%)': r.expected_return_annual * 100,
        'Slippage (bps)': r.slippage_cost_bps,
        'Net Return (%)': r.net_return_annual * 100,
        'Sharpe Ratio': r.sharpe_ratio,
        'Capital Required ($M)': r.capital_required / 1e6,
        'Capacity Util (%)': r.capacity_utilization * 100
    }
    for r in btc_results
])

# Find kill capacity
btc_kill_capacity = btc_analyzer.find_kill_capacity()

print("=" * 60)
print("📊 BTC FUNDING ARBITRAGE CAPACITY ANALYSIS")
print("=" * 60)
print(f"\n🔴 KILL CAPACITY: ${btc_kill_capacity/1e6:.1f}M")
print(f"   (Position size where slippage wipes out funding edge)")
print("\n" + btc_df.to_string(index=False))

In [ ]:
# Analyze ETH and Altcoin markets
eth_analyzer = FundingArbCapacityAnalyzer(ETH_MARKET)
alt_analyzer = FundingArbCapacityAnalyzer(ALTCOIN_MARKET)

eth_kill = eth_analyzer.find_kill_capacity()
alt_kill = alt_analyzer.find_kill_capacity()

print("\n" + "=" * 60)
print("📊 KILL CAPACITY BY MARKET")
print("=" * 60)
print(f"\n   BTC:     ${btc_kill_capacity/1e6:>8.1f}M")
print(f"   ETH:     ${eth_kill/1e6:>8.1f}M")
print(f"   Altcoin: ${alt_kill/1e6:>8.1f}M")
print(f"\n   Total addressable capacity: ${(btc_kill_capacity + eth_kill + alt_kill)/1e6:.1f}M")

## 5. AUM-Based Allocation Planning

In [ ]:
def plan_allocation(aum_usd: float, markets: Dict[str, MarketConditions]) -> pd.DataFrame:
    """
    Plan optimal allocation across markets for given AUM.
    Uses proportional allocation based on kill capacity.
    """
    results = []
    
    # Calculate kill capacities
    kill_caps = {}
    for name, market in markets.items():
        analyzer = FundingArbCapacityAnalyzer(market)
        kill_caps[name] = analyzer.find_kill_capacity()
    
    total_kill = sum(kill_caps.values())
    
    for name, market in markets.items():
        analyzer = FundingArbCapacityAnalyzer(market)
        
        # Allocate proportionally to kill capacity (with safety margin)
        safety_margin = 0.5  # Use 50% of kill capacity max
        max_allocation = kill_caps[name] * safety_margin
        
        # Proportional allocation
        allocation_pct = kill_caps[name] / total_kill
        allocated = min(aum_usd * allocation_pct, max_allocation)
        
        # Analyze at this allocation
        result = analyzer.analyze_capacity([allocated])[0]
        
        results.append({
            'Market': name,
            'Kill Capacity ($M)': kill_caps[name] / 1e6,
            'Allocation ($M)': allocated / 1e6,
            'Allocation (%)': allocated / aum_usd * 100,
            'Net Return (%)': result.net_return_annual * 100,
            'Sharpe': result.sharpe_ratio,
            'Cap Util (%)': result.capacity_utilization * 100
        })
    
    return pd.DataFrame(results)

# Test allocations at different AUM levels
markets = {'BTC': BTC_MARKET, 'ETH': ETH_MARKET, 'ALT': ALTCOIN_MARKET}

print("\n" + "=" * 70)
print("📊 ALLOCATION PLANS BY AUM")
print("=" * 70)

for aum in [1_000_000, 5_000_000, 10_000_000, 50_000_000]:
    print(f"\n🏦 AUM: ${aum/1e6:.0f}M")
    print("-" * 70)
    allocation_df = plan_allocation(aum, markets)
    print(allocation_df.to_string(index=False))
    
    # Portfolio stats
    total_alloc = allocation_df['Allocation ($M)'].sum()
    weighted_return = (allocation_df['Allocation ($M)'] * allocation_df['Net Return (%)']).sum() / total_alloc
    print(f"\n   Total Deployed: ${total_alloc:.2f}M ({total_alloc/aum*1e6:.1f}% of AUM)")
    print(f"   Portfolio Net Return: {weighted_return:.1f}%")

## 6. Capacity vs Slippage Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Net Return vs Position Size
ax1 = axes[0, 0]
ax1.plot(btc_df['Position Size ($M)'], btc_df['Net Return (%)'], 'b-', linewidth=2, label='BTC')

eth_results = eth_analyzer.analyze_capacity(position_sizes.tolist())
eth_df = pd.DataFrame([{'Position Size ($M)': r.position_size_usd/1e6, 'Net Return (%)': r.net_return_annual*100} for r in eth_results])
ax1.plot(eth_df['Position Size ($M)'], eth_df['Net Return (%)'], 'g-', linewidth=2, label='ETH')

ax1.axhline(y=0, color='r', linestyle='--', linewidth=1)
ax1.axvline(x=btc_kill_capacity/1e6, color='b', linestyle=':', alpha=0.5)
ax1.axvline(x=eth_kill/1e6, color='g', linestyle=':', alpha=0.5)

ax1.set_xscale('log')
ax1.set_xlabel('Position Size ($M)')
ax1.set_ylabel('Net Annual Return (%)')
ax1.set_title('Net Return vs Position Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Slippage vs Position Size
ax2 = axes[0, 1]
ax2.plot(btc_df['Position Size ($M)'], btc_df['Slippage (bps)'], 'b-', linewidth=2, label='BTC')
ax2.plot(eth_df['Position Size ($M)'], [r.slippage_cost_bps for r in eth_results], 'g-', linewidth=2, label='ETH')

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Position Size ($M)')
ax2.set_ylabel('Round-Trip Slippage (bps)')
ax2.set_title('Slippage vs Position Size')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Sharpe Ratio vs Position Size
ax3 = axes[1, 0]
ax3.plot(btc_df['Position Size ($M)'], btc_df['Sharpe Ratio'], 'b-', linewidth=2, label='BTC')
ax3.plot(eth_df['Position Size ($M)'], [r.sharpe_ratio for r in eth_results], 'g-', linewidth=2, label='ETH')

ax3.axhline(y=0, color='r', linestyle='--', linewidth=1)
ax3.set_xscale('log')
ax3.set_xlabel('Position Size ($M)')
ax3.set_ylabel('Sharpe Ratio')
ax3.set_title('Sharpe Ratio vs Position Size')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Capital Efficiency
ax4 = axes[1, 1]
btc_efficiency = btc_df['Net Return (%)'] / (btc_df['Position Size ($M)'] * BTC_MARKET.margin_requirement * 2) * btc_df['Position Size ($M)']
ax4.fill_between(btc_df['Position Size ($M)'], 0, btc_df['Net Return (%)'].clip(lower=0), alpha=0.3, label='Profitable Zone')
ax4.fill_between(btc_df['Position Size ($M)'], btc_df['Net Return (%)'].clip(upper=0), 0, alpha=0.3, color='red', label='Loss Zone')
ax4.axvline(x=btc_kill_capacity/1e6, color='r', linestyle='--', linewidth=2, label=f'Kill: ${btc_kill_capacity/1e6:.1f}M')
ax4.axvline(x=btc_kill_capacity/1e6 * 0.5, color='g', linestyle='--', linewidth=2, label=f'Safe: ${btc_kill_capacity/1e6*0.5:.1f}M')

ax4.set_xscale('log')
ax4.set_xlabel('Position Size ($M)')
ax4.set_ylabel('Net Return (%)')
ax4.set_title('BTC Funding Arb: Capacity Zones')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('funding_arb_capacity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Charts saved to funding_arb_capacity.png")

## 7. Sensitivity Analysis

In [ ]:
def sensitivity_analysis(base_market: MarketConditions, param_name: str, multipliers: List[float]) -> pd.DataFrame:
    """
    Test how kill capacity changes with market parameter variations.
    """
    results = []
    
    for mult in multipliers:
        # Create modified market
        modified = MarketConditions(
            bid_ask_spread_bps=base_market.bid_ask_spread_bps * (mult if param_name == 'spread' else 1),
            order_book_depth_usd=base_market.order_book_depth_usd * (mult if param_name == 'depth' else 1),
            daily_volume_usd=base_market.daily_volume_usd * (mult if param_name == 'volume' else 1),
            avg_funding_rate_8h=base_market.avg_funding_rate_8h * (mult if param_name == 'funding' else 1),
            funding_rate_std=base_market.funding_rate_std,
            maker_fee=base_market.maker_fee,
            taker_fee=base_market.taker_fee,
            margin_requirement=base_market.margin_requirement,
            liquidation_buffer=base_market.liquidation_buffer
        )
        
        analyzer = FundingArbCapacityAnalyzer(modified)
        kill_cap = analyzer.find_kill_capacity()
        
        results.append({
            'Multiplier': f'{mult:.1f}x',
            'Kill Capacity ($M)': kill_cap / 1e6
        })
    
    return pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 SENSITIVITY ANALYSIS (BTC Market)")
print("=" * 60)

multipliers = [0.5, 0.75, 1.0, 1.5, 2.0]

for param in ['spread', 'depth', 'volume', 'funding']:
    print(f"\n{param.upper()} Sensitivity:")
    sens_df = sensitivity_analysis(BTC_MARKET, param, multipliers)
    print(sens_df.to_string(index=False))

## 8. Recommendations Export

In [ ]:
import json
import yaml

# Generate recommendations
recommendations = {
    'analysis_date': datetime.now().isoformat(),
    'kill_capacities': {
        'BTC': round(btc_kill_capacity / 1e6, 2),
        'ETH': round(eth_kill / 1e6, 2),
        'Altcoin': round(alt_kill / 1e6, 2),
        'total': round((btc_kill_capacity + eth_kill + alt_kill) / 1e6, 2)
    },
    'recommended_allocations': {
        '1M_AUM': {
            'BTC': round(min(1_000_000 * 0.7, btc_kill_capacity * 0.5) / 1e6, 2),
            'ETH': round(min(1_000_000 * 0.2, eth_kill * 0.5) / 1e6, 2),
            'Altcoin': round(min(1_000_000 * 0.1, alt_kill * 0.5) / 1e6, 2)
        },
        '5M_AUM': {
            'BTC': round(min(5_000_000 * 0.7, btc_kill_capacity * 0.5) / 1e6, 2),
            'ETH': round(min(5_000_000 * 0.2, eth_kill * 0.5) / 1e6, 2),
            'Altcoin': round(min(5_000_000 * 0.1, alt_kill * 0.5) / 1e6, 2)
        },
        '10M_AUM': {
            'BTC': round(min(10_000_000 * 0.7, btc_kill_capacity * 0.5) / 1e6, 2),
            'ETH': round(min(10_000_000 * 0.2, eth_kill * 0.5) / 1e6, 2),
            'Altcoin': round(min(10_000_000 * 0.1, alt_kill * 0.5) / 1e6, 2)
        }
    },
    'safety_guidelines': {
        'max_capacity_utilization': 0.5,
        'rebalance_threshold': 0.1,
        'min_funding_threshold': 0.0001,
        'max_single_market_allocation': 0.7
    },
    'market_assumptions': {
        'BTC': {
            'daily_volume_usd': BTC_MARKET.daily_volume_usd,
            'avg_funding_rate_8h': BTC_MARKET.avg_funding_rate_8h,
            'bid_ask_spread_bps': BTC_MARKET.bid_ask_spread_bps
        },
        'ETH': {
            'daily_volume_usd': ETH_MARKET.daily_volume_usd,
            'avg_funding_rate_8h': ETH_MARKET.avg_funding_rate_8h,
            'bid_ask_spread_bps': ETH_MARKET.bid_ask_spread_bps
        }
    }
}

# Save
with open('funding_arb_capacity_plan.yml', 'w') as f:
    yaml.dump(recommendations, f, default_flow_style=False, sort_keys=False)

with open('funding_arb_capacity_plan.json', 'w') as f:
    json.dump(recommendations, f, indent=2)

print("=" * 60)
print("📁 EXPORTED CAPACITY PLAN")
print("=" * 60)
print("\n• funding_arb_capacity_plan.yml")
print("• funding_arb_capacity_plan.json")
print("\n" + yaml.dump(recommendations, default_flow_style=False, sort_keys=False))

## 9. Summary & Action Items

In [ ]:
print("\n" + "=" * 60)
print("📋 CAPACITY PLANNING SUMMARY")
print("=" * 60)

print(f"""
🔑 KEY FINDINGS:

1. KILL CAPACITIES (where slippage > funding edge):
   • BTC:     ${btc_kill_capacity/1e6:.1f}M
   • ETH:     ${eth_kill/1e6:.1f}M  
   • Altcoin: ${alt_kill/1e6:.1f}M
   • Total:   ${(btc_kill_capacity + eth_kill + alt_kill)/1e6:.1f}M

2. RECOMMENDED SAFE CAPACITY (50% of kill):
   • BTC:     ${btc_kill_capacity*0.5/1e6:.1f}M
   • ETH:     ${eth_kill*0.5/1e6:.1f}M
   • Altcoin: ${alt_kill*0.5/1e6:.1f}M

3. SCALING GUIDELINES:
   • At 1% AUM ($100K on $10M): Full deployment OK
   • At 5% AUM ($500K on $10M): Watch BTC/ETH slippage
   • At 10% AUM ($1M on $10M): Consider multi-venue

4. RISK FACTORS:
   • Funding rate collapse (historical: -0.1% to +0.3%)
   • Liquidity crises (spreads 10x normal)
   • Exchange outages / withdrawal delays

⚠️  ACTION ITEMS:
   □ Set position size alerts at 80% of safe capacity
   □ Monitor daily volume changes (capacity scales with volume)
   □ Backtest with historical funding spikes/crashes
   □ Implement TWAP/VWAP for entries > $500K
   □ Review capacity quarterly or after major market events
""")

print("\n✅ Capacity planning notebook complete!")